## HW1 - generate block simulation data and solve for the parameters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import MatrixMethods as mm
from SingleImage import SingleImage
from ImageBlock import ImageBlock
from BlockSimulation import *
from scipy import linalg as la


# Set print options to suppress scientific notation
np.set_printoptions(suppress=True)

## Part 1 - Simulate block of images

In [ ]:
# define random seed
np.random.seed(0)

a = SimulateBlock(50, (36, 24), overlap=0.6 , num_images=2, tie_pattern='3 mid frame', control_pattern='random block',num_control_points=4, rotaions_sigma=10, altitude=1000)

block = a.simulate_block()

In [ ]:
ax = block.draw_block(anotate=True)
ax.axis('equal')
plt.show()

In [ ]:
block.describe_block()

## Part 2 - Bundle Adjustment

In [ ]:
img = block.images[0]
Compute3DRotationMatrix(*img.exteriorOrientationParameters[3:])

In [ ]:
# drawing A matrix
A = block.ComputeDesignMatrix()
plt.figure(figsize=(10,10))
plt.spy(A)
plt.grid()
plt.show()

In [ ]:
# drawing A matrix
N = np.dot(A.T, A)
plt.figure(figsize=(10,10))
plt.spy(N)
plt.grid()
plt.show()

In [ ]:
X = block.compute_variables_vector()

In [ ]:
X

In [ ]:
lb = block.create_lb_vector()
l0 = block.compute_observation_vector()

In [ ]:
lb = lb.astype(np.float64)

In [ ]:
len(lb)/2

In [ ]:
print(np.round(lb,3))
print()
# rounding l0 to 3 digits after the decimal point
print(np.round(l0,3))


In [ ]:
block.images[0].tie_points

In [ ]:
a = SimulateBlock(50, (36, 24), overlap=0.6 , num_images=2, tie_pattern='3 mid frame', control_pattern='random block',num_control_points=4, rotaions_sigma=10, altitude=1000, sigma_location_noise=100, sigma_rotation_noise=20)
block = a.simulate_block()
X,RMSE,sigmaX = block.BundleAdjustment(0.01,100)

In [ ]:
print(a.images[0].exteriorOrientationParameters, '\n')
print(a.images_noisy[0].exteriorOrientationParameters, '\n')
print(block.images[0].exteriorOrientationParameters)